# Used the trained embeddings to classify hate-speeches
This notebook creates a neural classifier.

### Input:
    - Word-embeddings.
    - Training data.

### Output:
    - A binary classifier.
    - Evaluation on test data.

## Import libraries

In [ ]:
# Imports
import re
import string
import json
import math
from datetime import datetime
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import Module
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.init import xavier_uniform_
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from nltk.corpus import stopwords

device = 'cuda'

import random

random.seed(26)
np.random.seed(62)
torch.manual_seed(2021)

## Load data

In [ ]:
# train data
train_df = pd.read_csv('save/hindi_train_preprocessed.csv')
train_sentences = [[int(s) for s in text.split()] for text in train_df['sentence']]
train_df['hate'] = (train_df['task_1'] == 'HOF').astype(int)
train_labels = train_df['hate'].to_numpy()

# test data
test_df = pd.read_csv('save/hindi_test_preprocessed.csv')
test_sentences = [[int(s) for s in text.split()] for text in test_df['sentence']]
test_df['hate'] = (test_df['task_1'] == 'HOF').astype(int)
test_labels = test_df['hate'].to_numpy()

# word <-> convertion
with open('save/hindi_word_to_int_dict.json', 'r') as f:
    word_to_int = json.load(f)
with open('save/hindi_int_to_word_dict.json', 'r') as f:
    int_to_word = json.load(f)
    int_to_word = {int(k) : v for k, v in int_to_word.items()}

# word-counter
with open('save/hindi_word_counter.json', 'r') as f:
    word_counter = json.load(f)
    
vocab_size = len(word_to_int)
print('vocab size:', vocab_size)

In [ ]:
print('sample data:')
print('train:')
print(train_sentences[:2])
print(train_labels[:2])
print('test:')
print(test_sentences[:2])
print(test_labels[:2])

## Hyper-parameters

In [ ]:
embedding_path = 'save/hindi_embedding_weights_10_negfac.pt'
# embedding_path = 'save/hindi_embedding_weights_3_wsize_10_negfac.pt'
embedding_size = 300
att_dim = 150
# lstm_dim = 100
# lstm_bidirectional = True
learning_rate = 5e-5
batch_size = 32
epochs = 30

## Build datasets

In [ ]:
# augmentation by removing some random words from a sentence
augmentation_rate = 0. # value range: [0, 1]. Set to 0. to disable augmentation.
sample_prop = (0.7, 0.95)

class HOFDataset(Dataset):
    def __init__(self, sentences, labels, do_aug):
        self.data = []
        if do_aug:
            augmentation = np.random.uniform(low=0, high=1, size=len(sentences)) < augmentation_rate
        else:
            augmentation = np.zeros(len(sentences))
        subsample = np.random.uniform(low=sample_prop[0], high=sample_prop[1], size=len(sentences))
        for sentence, label, aug, sub in zip(sentences, labels, augmentation, subsample):
            if len(sentence):
                if aug:
                    length = len(sentence)
                    selected_indices = np.random.choice(range(length), math.ceil(length*sub), replace=False)
                    selected_indices.sort()
                    aug_sentence = torch.tensor(sentence, dtype=torch.long)[selected_indices]
                    aug_label = 0.5 + (label - 0.5)*sub
                    self.data.append(
                        (aug_sentence, 
                         torch.tensor(aug_label, dtype=torch.float))
                    )
                else:
                    self.data.append(
                        (torch.tensor(sentence, dtype=torch.long), 
                         torch.tensor(label, dtype=torch.float))
                    )
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]
    
def preprocess_batch(batch):
    texts, labels = list(zip(*batch))
    seq_lens = torch.tensor([len(text) for text in texts], dtype=torch.long)
    texts = pad_sequence(texts, padding_value=0)
    labels = torch.tensor(labels).unsqueeze(1)

    seq_lens, sorted_idx = seq_lens.sort(descending=True)
    texts = texts[:,sorted_idx]
    labels = labels[sorted_idx]
    return texts, seq_lens, labels

train_dataset = HOFDataset(train_sentences, train_labels, do_aug=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True, collate_fn=preprocess_batch,
                          num_workers=2)

test_dataset = HOFDataset(test_sentences, test_labels, do_aug=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                         shuffle=False, collate_fn=preprocess_batch)

## Network architecture

### Positional encoding

In [ ]:
max_len = 100

def get_pe(p, i):
    k = i // 2
    if i % 2 == 0:
        return np.sin(p/(10000**(2*k/embedding_size)))
    else:
        return np.cos(p/(10000**(2*k/embedding_size)))

pe = np.zeros((embedding_size, max_len))
for p in range(max_len):
    for i in range(embedding_size):
        pe[i, p] = get_pe(p, i)
        
fig, ax = plt.subplots(figsize=(8, 5))
ax.imshow(pe.T[:,:512])
ax.set_title('Positional encoding')
ax.set_xlabel('dimension')
ax.set_ylabel('position')

plt.show()

pe = pe.T # for easier addition in the future operations
pe = torch.tensor(pe, requires_grad=False).to(device)

### Attention mask

In [ ]:
def mask_seq(seq_lens):
    mask = torch.zeros((len(seq_lens), max(seq_lens))).bool()
    for i, seq_len in enumerate(seq_lens):
        mask[i, seq_len:] = True
    return mask.to(device)

### Encoder layer

In [ ]:
class Encoder(Module):
    def __init__(self, embed_dim, num_heads, dropout, batch_first, norm_dim):
        super(Encoder, self).__init__()
        
        self.batch_first = batch_first
        self.norm_dim = norm_dim
        self.att = nn.MultiheadAttention(embed_dim=embed_dim,
                                         num_heads=num_heads,
                                         dropout=dropout,)
        self.fc = nn.Linear(embed_dim, embed_dim)
        self.fc.weight = xavier_uniform_(self.fc.weight)
    
    def forward(self, inp, pad_mask):
        # make batch second
        if self.batch_first:
            inp = inp.permute(1, 0, 2)        
        
        # inp, att_out: [seq dim, batch dim, embed dim]
        att_out, _ = self.att(inp, inp, inp, key_padding_mask=pad_mask)
        
        # out, fc_out: [batch dim, seq dim, embed dim]
        out = (att_out + inp).permute(1, 0, 2)
        out = F.layer_norm(out, out.shape[self.norm_dim:])
        out = F.layer_norm(self.fc(out) + out, out.shape[self.norm_dim:])
        
        # restore the original shape
        if not self.batch_first:
            out = out.permute(1, 0, 2)
        return out

### Network architecture

In [ ]:
class Classifier(Module):
    def __init__(self):
        super(Classifier, self).__init__()

        self.embed = nn.Embedding(vocab_size, embedding_size)
        self.embed.load_state_dict(torch.load(embedding_path, map_location=torch.device(device)))
        self.embed.requires_grad = False
        
        self.encoder_1 = Encoder(
            embed_dim=embedding_size,
            num_heads=10,
            dropout=0.7,
            batch_first=False,
            norm_dim=1,
        )
        self.encoder_2 = Encoder(
            embed_dim=embedding_size,
            num_heads=5,
            dropout=0.7,
            batch_first=False,
            norm_dim=1,
        )

        self.fc = nn.Linear(embedding_size, 1)

    def forward(self, inp, seq_lens):
        # add positional encoding
        # out: [seq dim, batch dim, embed dim]
        out = self.embed(inp)
        for i in range(out.shape[1]):
            out[:, i, :] += pe[:out.shape[0], :]
        # padding mask
        pad_mask = mask_seq(seq_lens)
        # encoder
        out = self.encoder_1(out, pad_mask)
        out = self.encoder_2(out, pad_mask)
        # predict
        out = self.fc(out)
        out = out.squeeze(2)
        pred = torch.zeros((out.size(1), 1))
        for i, seq_len in enumerate(seq_lens):
            pred[i, 0] = out[:seq_len, i].mean()
        return pred

clf = Classifier().to(device)
torch.save(clf.state_dict(), 'save/hindi_hindi_initial.pt')

### Loss function and optimizer

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(clf.parameters(), lr=learning_rate)

### Measure performance on test data

In [ ]:
def predict_test():
    losses = 0
    acc_cnt = 0
    cnt = 0
    preds = []
    true_labels = []
    clf.eval()
    for texts, seq_lens, labels in test_loader:
        pred = clf(texts.to(device), seq_lens).detach().to('cpu')
        loss = criterion(pred, labels)
        losses += loss.detach().item() * len(texts)
        acc_cnt += sum((pred > 0) == (labels > 0)).item()
        preds.extend(pred.view(-1))
        true_labels.extend(labels.view(-1))
        cnt += texts.size(1)
    
    preds = np.array(preds) > 0
    macro_f1 = f1_score(true_labels, preds, average='macro')
    weighted_f1 = f1_score(true_labels, preds, average='weighted')
    return losses / cnt, acc_cnt / cnt, macro_f1, weighted_f1

## Training

In [ ]:
clf.load_state_dict(torch.load('save/hindi_hindi_initial.pt'))

list_test_acc = []
early_stop = 3

for epoch in range(1, epochs + 1):
    losses = 0.
    acc_cnt = 0
    cnt = 0
    clf.train()
    for texts, seq_lens, labels in tqdm(train_loader):
        optimizer.zero_grad()
        pred = clf(texts.to(device), seq_lens)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        losses += loss.detach().item() * len(texts)
        acc_cnt += sum((pred.to('cpu') > 0) == (labels > 0)).item()
        cnt += texts.size(1)

    epoch_loss = losses / cnt
    epoch_acc = acc_cnt / cnt
    test_loss, test_acc, test_macro_f1, test_weighted_f1 = predict_test()
    print(f'Epoch {epoch:2}: Train loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f}. '
        f'Test loss: {test_loss:.4f}, acc: {test_acc:.4f}, '
        f'macro_f1: {test_macro_f1:.4f}, weighted_f1: {test_weighted_f1:.4f}',
        flush=True)

    list_test_acc.append(test_acc)
    if len(list_test_acc) > early_stop and max(list_test_acc[-early_stop:]) <= max(list_test_acc[:-early_stop]):
        print(f'Early stopping: test accuracy does not increase after {early_stop} epochs')
        break

In [ ]:
# save final model
torch.save(clf.state_dict(), 'save/hindi_hindi_final_clf.pt')

## Batch statistics

In [ ]:
def train(epochs):
    # restore model to untrained
    clf.load_state_dict(torch.load('save/hindi_hindi_initial.pt'))
    
    # store test accuracy and macro-f1
    list_test_acc = []
    list_test_macro_f1 = []
    
    # train
    for epoch in range(1, epochs + 1):
        losses = 0.
        acc_cnt = 0
        cnt = 0
        clf.train()
        for texts, seq_lens, labels in train_loader:
            optimizer.zero_grad()
            pred = clf(texts.to(device), seq_lens)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()
            losses += loss.detach().item() * len(texts)
            acc_cnt += sum((pred.to('cpu') > 0) == (labels > 0)).item()
            cnt += texts.size(1)

        epoch_loss = losses / cnt
        epoch_acc = acc_cnt / cnt
        test_loss, test_acc, test_macro_f1, test_weighted_f1 = predict_test()

        list_test_acc.append(test_acc)
        list_test_macro_f1.append(test_macro_f1)
        
    return list_test_acc, list_test_macro_f1

def batch_train(ntime, epochs):
    list_test_acc = []
    list_test_macro_f1 = []
    for i in range(ntime):
        acc, macro_f1 = train(epochs=epochs)
        list_test_acc.append(acc)
        list_test_macro_f1.append(macro_f1)
        print(f'Trained {i+1} time(s)')
    
    return list_test_acc, list_test_macro_f1

def plot_results(list_test_acc, list_test_macro_f1):
    # accuracy plot
    mean_acc = [np.mean([l[i] for l in list_test_acc if len(l) > i]) for i in range(max(len(li) for li in list_test_acc))]

    fig = go.Figure()
    for i, acc_curve in enumerate(list_test_acc):
        fig.add_trace(go.Scatter(
            x=list(range(len(acc_curve))), 
            y=acc_curve,
            mode='lines',
            name=f'model {i+1}',
            opacity=0.3,
        ))
    fig.add_trace(go.Scatter(
        x=list(range(len(mean_acc))), 
        y=mean_acc,
        mode='lines+markers',
        line_color='blue',
        name='mean',
    ))

    fig.update_layout(
        title='Accuracy plot',
        xaxis_title='Epoch',
        yaxis_title='Accuracy'
    )

    fig.show()

    # macro-f1 plot
    mean_mac = [np.mean([l[i] for l in list_test_macro_f1 if len(l) > i]) for i in range(max(len(li) for li in list_test_macro_f1))]

    fig = go.Figure()
    for i, mac_curve in enumerate(list_test_macro_f1):
        fig.add_trace(go.Scatter(
            x=list(range(len(mac_curve))), 
            y=mac_curve,
            mode='lines',
            name=f'model {i+1}',
            opacity=0.3,
        ))
    fig.add_trace(go.Scatter(
        x=list(range(len(mean_mac))), 
        y=mean_mac,
        mode='lines+markers',
        line_color='blue',
        name='mean',
    ))

    fig.update_layout(
        title='Macro-f1 plot',
        xaxis_title='Epoch',
        yaxis_title='Macro-f1'
    )

    fig.show()

In [ ]:
list_test_acc, list_test_macro_f1 = batch_train(ntime=5, epochs=15)
plot_results(list_test_acc, list_test_macro_f1)